In [82]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import cma

from bayes_optim import BO
from bayes_optim.search_space import RealSpace
from bayes_optim.surrogate import GaussianProcess, trend

# only noiseless functions
from benchmark.bbobbenchmarks import F1, F2, F3, F4, F5, F6, F7, F8, F9, F10, F11, F12, F13, F14, F15, F16, F21

In [101]:
dim = 2
lb, ub = -5, 5

space = RealSpace([lb, ub]) * dim

mean = trend.constant_trend(dim, beta=None)
thetaL = 1e-10 * (ub - lb) * np.ones(dim)
thetaU = 10 * (ub - lb) * np.ones(dim)
theta0 = np.random.rand(dim) * (thetaU - thetaL) + thetaL

doe_size = 5
max_FEs = 5
verbose = False

n_test_samples = 1000

In [102]:
def regret_function(obj_func, doe_size):
    def func(X):
        _X = np.split(X, doe_size)
        model = GaussianProcess(
            mean=mean,
            corr="squared_exponential",
            theta0=theta0,
            thetaL=thetaL,
            thetaU=thetaU,
            nugget=0,
            noise_estim=False,
            optimizer="BFGS",
            wait_iter=3,
            random_start=dim,
            likelihood="concentrated",
            eval_budget=100 * dim,
        )
        opt = BO(
            search_space=space,
            obj_fun=obj_func,
            model=model,
            DoE_size=doe_size,
            acquisition_fun="EI",
            initial_points=_X,
            max_FEs=max_FEs,
            verbose=verbose,
            n_point=1
        )
        x, value, _ = opt.run()
        return value - obj_func.getfopt()
    return func

In [113]:
obj_func = F21(1)
regret = regret_function(obj_func, doe_size)

In [114]:
xopt, es = cma.fmin2(regret, np.zeros(2 * doe_size), 0.5)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=98496, Mon Oct 17 13:13:45 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 6.526985214870777e+00 1.0e+00 4.82e-01  5e-01  5e-01 0:00.3
    2     20 2.011434718095828e+00 1.2e+00 5.07e-01  5e-01  5e-01 0:00.6
    3     30 1.986158151215854e+00 1.3e+00 5.29e-01  5e-01  6e-01 0:00.8
   22    220 2.682153071662086e-01 2.3e+00 9.77e-01  8e-01  1e+00 0:03.8
   43    430 3.555628307971404e-03 3.7e+00 3.66e-01  2e-01  5e-01 0:07.8
   69    690 4.067698286291943e-05 1.2e+01 2.70e-01  1e-01  4e-01 0:13.2
   93    930 3.558730270469823e-07 2.5e+01 1.53e-01  3e-02  2e-01 0:19.3
  100   1000 1.378381853101018e-10 2.8e+01 8.54e-02  2e-02  1e-01 0:20.4
  142   1420 2.348343741687131e-11 1.4e+02 7.55e-02  5e-03  1e-01 0:28.4
  196   1960 0.000000000000000e+00 7.8e+02 3.06e-02  4e-04  6e-02 0:36.9
termination on tolfun=1e-11 (Mon Oct 17 13:14:22 2022)
final/bestever f-value = 0.000000e+00 0.000000e+00
i

In [115]:
obj_func._xopt

array([-2.51487651, -1.78747656])

In [116]:
np.split(xopt, doe_size)

[array([-0.19516681,  2.00839629]),
 array([ 0.33629866, -2.05740294]),
 array([-2.02113167, -1.37511151]),
 array([-2.51483613, -1.78743351]),
 array([0.32704453, 3.21379349])]

In [111]:
model = GaussianProcess(
        mean=mean,
        corr="squared_exponential",
        theta0=theta0,
        thetaL=thetaL,
        thetaU=thetaU,
        nugget=0,
        noise_estim=False,
        optimizer="BFGS",
        wait_iter=3,
        random_start=dim,
        likelihood="concentrated",
        eval_budget=100 * dim,
    )
opt = BO(
    search_space=space,
    obj_fun=obj_func,
    model=model,
    DoE_size=doe_size,
    acquisition_fun="EI",
    initial_points=np.split(xopt, doe_size),
    max_FEs=max_FEs,
    verbose=verbose,
    n_point=1
)

In [112]:
opt.run()

([0.2528000084293766, -1.1568000321896277], array([79.48]), {'max_FEs': 5})